# Dog vs. Cat Image Classifier using VGGNet

## Imports

In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf

## Constants

In [ ]:
TRAIN_DATA_DIR = "/Users/dayanarios/Education/MachineLearning/HW/VGGNet-Dog-vs-Cat/data/train/"
TEST_DATA_DIR = "/Users/dayanarios/Education/MachineLearning/HW/VGGNet-Dog-vs-Cat/data/test1/"
IMG_WIDTH = 150
IMG_HEIGHT = 150

TRAIN_DATA_DIR and TEST_DATA_DIR contain the path to the training and testing data directories respectively. Both training and testing datasets where obtained from [Kaggle's Dog vs Cats 2013 Competition](https://www.kaggle.com/c/dogs-vs-cats/data). The original set contains 25,000 training images and 12,500 testing images. It should be noted the training set was split into 80% training and 20% validation. 

IMG_WIDTH and IMG_HEIGHT contained the fixed dimension for image width and height respectively. 

## Image Preprocessing

Each image in our dataset follows the notation cat.x.jpg or dog.x.jpg where x is some integer between 0 and 12,499, inclusive. Images must be preprocessed and formatted in order to facilitate training and testing. Cat images are labeled as 0 and dog images as 1. 

### 1. Preparing the Training Data

In [ ]:
def label_img(label):  
    if label == 'cat':
        return 0
    elif label == 'dog':
        return 1
    else:
        return -1

The label_img method takes and a string as an argument and compares it with the strings 'cat' and 'dog'. Each cat image is assigned the label 0 and each dog image is assigned 1. If the string passed to this method contains a label that differs from our given notation, the label_img function returns -1. 

In [ ]:
def prepare_training_data():
    training_data = []
    for img in os.listdir(TRAIN_DATA_DIR):
        if img.endswith(".jpg"):
            path = os.path.join(TRAIN_DATA_DIR, img)
            label, _id, _ = img.split(".", 2)
            label_code = label_img(label)
            img = cv2.imread(path,cv2.IMREAD_COLOR)
            img = cv2.resize(img, (IMG_WIDTH,IMG_HEIGHT))
            training_data.append((label_code, img, _id))
    
    training_data = np.array(training_data)
    #np.save('training_data', training_data)
    return training_data

Our program takes the file name of each image and assigns a label to it. training_data will contain all the information necessary for training our classifier. Line 3 iterates through the given directory and stores the fliename in img. To ensure we observe only .jpg files each filename's extension is compared with the desired extension. The filename is then split at each "." and stored in their respective variables. Label contains the type of image, namely 'cat' or 'dog', while \_id contains the image's assigned id. The extension of the filename is placed in the throwaway variable '_'. 

The images label is passed to the label_img method which returns 0 for a cat image and 1 for a dog image. 

The VGGNet model assumes inputs are square images. To ensure there are no discrepancies between picture dimensions all images are resized to 150 x 150. The cv2 package handles all image processing. cv2.imread(path,cv2.IMREAD_COLOR) reads the image found at the given image path, the second arguments how the image should be processed. In the contents of this project we want images to retain their color properties thus we use cv2.IMREAD_COLOR. (It should be noted that cv2 uses BGR instead of RGB as the default color ordering)

training_data array will contain all the data for training. Each element will consist of a tuple containing the assigned label code and an img object. training_data is converted into a numpy array in order to faciliate linear algebra operations involved in VGGNet. 

In [ ]:
def display_image(data, n):
    #data = np.load('training_data.npy', allow_pickle=True)
    image = data[n][1]
    plt.axis("off")
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.show()

display_image displays the $n^{th}$ image in the given training dataset. This function is used to ensure the data from prepare_training_data() was properly processed using the cv2 package. 

### 2. Preparing Test Data
Testing data has the format x.jpg where x is an integer between 1 and 12,500. Each image must be accurately classified as either a dog or cat image.  

In [ ]:
def prepare_testing_data():
    testing_data = []
    for img in os.listdir(TEST_DATA_DIR):
        if img.endswith(".jpg"):
            path = os.path.join(TEST_DATA_DIR, img)
            print(path)
            _id, _ = img.split(".", 1)
            print(_id)
            img = cv2.imread(path,cv2.IMREAD_COLOR)
            img = cv2.resize(img, (IMG_WIDTH,IMG_HEIGHT))
            testing_data.append((img, _id))
    
    #cv2.imshow("test", img)
    #cv2.waitKey(25)
    #cv2.destroyAllWindows()
    testing_data = np.array(testing_data)
    return testing_data

Testing Data Needs to be prepared in a similar fashion to training data. All images are read using the cv2 package, resized accordingly, and appended to our testing data set, testing_data. Each entry is stored in a tuple with two elements, the first contains the img object and the second the image id, extracted from the image's filename. testing_data is converted to a numpy array for later linear algebra calculations. 

## VGGNET Architecture

VGGNet Architecture with 16 layers will be implemented using Keras with Tensorflow running in the backend. 

In [ ]:
if __name__ == '__main__':
    prepare_training_data()